In [1]:
import yaml
import salt.client
import salt.config                                                      
opts = salt.config.minion_config('/etc/salt/minion')
opts['file_client'] = 'local'

In [2]:
opts['grains'] = yaml.safe_load('''
host: minion
id: minion.domain.net
domain: domain.net
fqdn: minion.domain.net
fqdn_ip4:
  - 192.168.1.3
saltversion: 2016.3.3
os: Debian
os_family: Debian
osarch: amd64
oscodename: jessie
osfinger: Debian-8
osfullname: Debian
osmajorrelease: '8'
osrelease: '8.2'
biosreleasedate: 01/01/2011
biosversion: Bochs
cpuarch: x86_64
ip4_nameservers:
  - 8.8.8.8
  - 8.8.4.4
ip4_interfaces:
  eth0:
    - 192.168.1.3
kernel: Linux
localhost: minion.domain.net
mem_total: 2010
num_cpus: 2
''')

In [3]:
opts['pillar'] = yaml.safe_load('''
secret: 1A2B3C4D5E6F
ns:
  - 12.34.45.1
  - 12.34.45.2
''')

In [4]:
state = '''
{% set secret = pillar['secret'] %}
{% set domain = grains['domain'] %}
{% set osfinger = salt['grains.get']('osfinger') %}
{% set nameservers = salt['pillar.get']('ns') %}

{% if domain == 'domain.net' %}
set secret:
  file.managed:
    - name: /etc/secret
    - source: salt://state/secret
    - template: jinja
    - context:
        secret: {{ secret }}
{% endif %}

eth0:
  network.managed:
    - dns:
        {% for ip in nameservers %}
      - {{ ip }}
        {% endfor %}
'''

In [5]:
caller = salt.client.Caller(mopts=opts)

In [6]:
print(caller.cmd('slsutil.renderer', string=state, default_renderer='jinja').strip())

eth0:
  network.managed:
    - dns:
        
      - 12.34.45.1
        
      - 12.34.45.2
